In [1]:
import numpy as np
import cv2
import sys
import os
sys.path.insert(0, os.path.abspath('..'))

import salientregions as sr

In [2]:
#Load the image
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/_original.jpg'
#path_to_image = '../../../Data/OxFrei/09_small_palace/_original.jpg'
original = cv2.imread(path_to_image)
sr.show_image(original)
path_to_image = '/home/dafne/SalientRegions/LargeScaleImaging/Data/OxFrei/01_graffiti/JPG/viewpoint1.jpg'
viewpoint = cv2.imread(path_to_image)
sr.show_image(viewpoint)

In [3]:
#Convert to grey scale
grayscale = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
grayscale1 = cv2.cvtColor(viewpoint, cv2.COLOR_BGR2GRAY)
sr.show_image(grayscale)
sr.show_image(grayscale1)

In [4]:
#Binarize according to threshold
ret, binarized = cv2.threshold(grayscale, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
ret, binarized1 = cv2.threshold(grayscale1, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
sr.show_image(binarized)
sr.show_image(binarized1)

In [5]:
regions = sr.get_salient_regions(binarized, vizualize=False)
holes, islands, indents, prots = regions['holes'], regions['islands'], regions['indentations'], regions['protrusions']
regions1 = sr.get_salient_regions(binarized1, vizualize=False)
holes1, islands1, indents1, prots1 = regions1['holes'], regions1['islands'], regions1['indentations'], regions1['protrusions']

In [6]:
sr_vizualized = sr.vizualize_elements(grayscale, holes, islands, indents, prots)
sr_vizualized_holes = sr.vizualize_elements(grayscale1, holes1, islands1, indents1, prots1)
print ''

In [ ]:
#Get keypoints for the salient regions, from cc
def get_keypoints(mask):
    img, contours, hierarchy = cv2.findContours(mask,cv2.RETR_LIST ,cv2.CHAIN_APPROX_SIMPLE)
    kps = []
    for cont in contours:
        (x,y), r = cv2.minEnclosingCircle(cont)
        kps.append(cv2.KeyPoint(x, y, _size=1.5*2*r))
    return kps

In [ ]:
#Get Keypoints for our salient regions
sr_kps = get_keypoints(holes) #+ get_keypoints(islands) + get_keypoints(indents) + get_keypoints(prots)
sr_kps1 = get_keypoints(holes1)# + get_keypoints(islands1) + get_keypoints(indents1) + get_keypoints(prots1)

In [ ]:
im_with_keypoints = cv2.drawKeypoints(sr_vizualized, sr_kps, np.array([]), (0,0,255), cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sr.show_image(im_with_keypoints, 'keypoints', 100000)

In [ ]:
#Try out MSER
vis =  cv2.cvtColor(grayscale.copy(), cv2.COLOR_GRAY2BGR)
mser = cv2.MSER_create()
regions = mser.detectRegions(grayscale, None)
region_kps = mser.detect(grayscale)
region_kps1 = mser.detect(grayscale1)
hulls = [cv2.convexHull(p.reshape(-1, 1, 2)) for p in regions]
cv2.polylines(vis, hulls, 1, (0, 255, 0), 1)
sr.show_image(vis, 'MSER regions')

In [ ]:
kp_mser = region_kps[0]
region_mser = regions[0]
(x,y), r = cv2.minEnclosingCircle(region_mser)
print kp_mser.size
print kp_mser.pt
print x, y, r

In [ ]:
orb = cv2.ORB_create()
orb_kps, orb_des = orb.compute(original, sr_kps)
orb_kps1, orb_des1 = orb.compute(viewpoint, sr_kps1)

In [ ]:
#Matching!
matcher = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)
matches = matcher.match(orb_des, orb_des1)
matches = sorted(matches, key = lambda x:x.distance)
img3 = cv2.drawMatches(grayscale,orb_kps,grayscale1,orb_kps1,matches[:10], 
                       None, singlePointColor=[128,128,128],flags=cv2.DRAW_MATCHES_FLAGS_NOT_DRAW_SINGLE_POINTS)

sr.show_image(img3, 'matches', 100000)

In [ ]:
im_with_keypoints = cv2.drawKeypoints(grayscale, sr_kps, np.array([]), (0,0,255), 
                                      cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
sr.show_image(im_with_keypoints, 'keypoints', 100000)